In [61]:
import pandas as pd
import numpy as np

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

In [62]:
data = pd.read_csv('soybean.csv')
data.shape

(683, 36)

In [63]:
data.head()

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             683 non-null    object
 1   plant-stand      683 non-null    object
 2   precip           683 non-null    object
 3   temp             683 non-null    object
 4   hail             683 non-null    object
 5   crop-hist        683 non-null    object
 6   area-damaged     683 non-null    object
 7   severity         683 non-null    object
 8   seed-tmt         683 non-null    object
 9   germination      683 non-null    object
 10  plant-growth     683 non-null    object
 11  leaves           683 non-null    object
 12  leafspots-halo   683 non-null    object
 13  leafspots-marg   683 non-null    object
 14  leafspot-size    683 non-null    object
 15  leaf-shread      683 non-null    object
 16  leaf-malf        683 non-null    object
 17  leaf-mild        683 non-null    ob

In [65]:
x = data.drop('class', axis=1)
y = data['class']
print('X: {}, Y: {}'.format(x.shape, y.shape))

X: (683, 35), Y: (683,)


In [66]:
y = pd.get_dummies(y)
y.shape

(683, 19)

In [67]:
ordinal = ['date', 'crop-hist', 'severity', 'germination', 'leafspot-size']
nominal = ['plant-stand', 'precip', 'temp', 'hail', 'area-damaged', 'seed-tmt', 'plant-growth', 'leaves', 'leafspots-halo', 
           'leafspots-marg', 'leaf-shread', 'leaf-malf', 'leaf-mild', 'stem', 'lodging', 'stem-cankers', 'canker-lesion', 
           'fruiting-bodies', 'external-decay', 'mycelium', 'int-discolor', 'sclerotia', 'fruit-pods', 'fruit-spots', 
          'seed', 'mold-growth', 'seed-discolor', 'seed-size', 'shriveling', 'roots']
print(len(ordinal) + len(nominal))

35


In [68]:
x[ordinal]

,date,crop-hist,severity,germination,leafspot-size
0,october,same-lst-yr,pot-severe,90-100,dna
1,august,same-lst-two-yrs,severe,80-89,dna
2,july,same-lst-yr,severe,lt-80,dna
3,july,same-lst-yr,severe,80-89,dna
4,october,same-lst-two-yrs,pot-severe,lt-80,dna
...,...,...,...,...,...
678,april,?,?,?,dna
679,april,diff-lst-year,?,?,dna
680,june,diff-lst-year,?,?,dna
681,april,same-lst-yr,?,?,gt-1/8


In [69]:
# Aplica Label Encoder as colunas ordinais
for k in ordinal:
    labelEncoder = LabelEncoder()
    x[k] = labelEncoder.fit_transform(x[k])
x.head()

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,int-discolor,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots
0,6,normal,gt-norm,norm,yes,4,low-areas,2,none,1,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
1,2,normal,gt-norm,norm,yes,3,scattered,3,fungicide,0,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
2,3,normal,gt-norm,norm,yes,4,scattered,3,fungicide,3,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
3,3,normal,gt-norm,norm,yes,4,scattered,3,none,0,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
4,6,normal,gt-norm,norm,yes,3,scattered,2,none,3,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm


In [70]:
one_hot = make_column_transformer((OneHotEncoder(categories='auto', sparse=False), nominal), remainder='passthrough')
x = pd.DataFrame(one_hot.fit_transform(x))
x.head()

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,6.0,4.0,2.0,1.0,1.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,3.0,3.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,4.0,3.0,3.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,4.0,3.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,6.0,3.0,2.0,3.0,1.0


In [71]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=0)

In [75]:
model = Sequential()
model.add(Dense(units=58, input_dim=113))
model.add(Dense(units=30))
model.add(Dense(units=19, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
model.fit(x_treino, y_treino, epochs=300)

Epoch 1/300
15/15 [==============================] - 1s 2ms/step - loss: 0.6284 - accuracy: 0.0478
Epoch 2/300
15/15 [==============================] - 0s 2ms/step - loss: 0.2817 - accuracy: 0.0994
Epoch 3/300
15/15 [==============================] - 0s 2ms/step - loss: 0.1778 - accuracy: 0.3248
Epoch 4/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1537 - accuracy: 0.3733
Epoch 5/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1357 - accuracy: 0.5163
Epoch 6/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1201 - accuracy: 0.6144
Epoch 7/300
15/15 [==============================] - 0s 2ms/step - loss: 0.1130 - accuracy: 0.6342
Epoch 8/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1014 - accuracy: 0.7135
Epoch 9/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0932 - accuracy: 0.7136
Epoch 10/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0844 - accuracy: 0.7733
Epoch 11/

15/15 [==============================] - 0s 2ms/step - loss: 0.0196 - accuracy: 0.9261
Epoch 83/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9374
Epoch 84/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 0.9598
Epoch 85/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9294
Epoch 86/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 0.9305
Epoch 87/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 0.9207
Epoch 88/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 0.9396
Epoch 89/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0198 - accuracy: 0.9171
Epoch 90/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 0.9326
Epoch 91/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.9261
Epoch 92/300


15/15 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 0.9386
Epoch 164/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.9420
Epoch 165/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.9343
Epoch 166/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9343
Epoch 167/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 0.9429
Epoch 168/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 0.9561
Epoch 169/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0149 - accuracy: 0.9308
Epoch 170/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9369
Epoch 171/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0159 - accuracy: 0.9247
Epoch 172/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 0.9507
Epoc

15/15 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 0.9516
Epoch 245/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 0.9341
Epoch 246/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.9337
Epoch 247/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.9224
Epoch 248/300
15/15 [==============================] - 0s 5ms/step - loss: 0.0133 - accuracy: 0.9496
Epoch 249/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0115 - accuracy: 0.9430
Epoch 250/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 0.9343
Epoch 251/300
15/15 [==============================] - 0s 4ms/step - loss: 0.0170 - accuracy: 0.9184
Epoch 252/300
15/15 [==============================] - 0s 4ms/step - loss: 0.0135 - accuracy: 0.9499
Epoch 253/300
15/15 [==============================] - 0s 4ms/step - loss: 0.0164 - accuracy: 0.9136
Epoc

Resultado com 113 Na camada de entrada, 58 na primeira camada oculta, 30 na segunda camada oculta, 19 na camada de saida 
e 300 epocas: 95%

<img src='rede.png'>

In [77]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 58)                6612      
_________________________________________________________________
dense_10 (Dense)             (None, 30)                1770      
_________________________________________________________________
dense_11 (Dense)             (None, 19)                589       
Total params: 8,971
Trainable params: 8,971
Non-trainable params: 0
_________________________________________________________________
